
# Carga y Consolidación de CSV por subcarpetas **LS1..LS6**

Este notebook:
- Detecta subcarpetas cuyo nombre empieza por `LS` (LS1..LS6).
- Lee **todos** los `.csv` dentro de cada subcarpeta (recursivo).
- Crea un DataFrame por subcarpeta: `df_ls1`, `df_ls2`, ... (solo si existen).
- Expone también `dfs_por_ls` (dict) y guarda opcionalmente a **Parquet**.

> **Ruta esperada:** por defecto, el directorio `data_drive` debe estar en la misma carpeta que este notebook.  
> Alternativa: sube un ZIP llamado `data_drive.zip` junto a este notebook; el notebook lo descomprimirá automáticamente.


In [1]:

# %% [setup]
from __future__ import annotations
import logging
import re
from pathlib import Path
from typing import Dict, List, Optional

import pandas as pd

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S",
)

# Selección de engine de lectura para pandas
_ENGINE: str = "python"
try:
    import pyarrow  # noqa: F401
    _ENGINE = "pyarrow"
except Exception:
    _ENGINE = "python"

print(f"Engine de lectura seleccionado: {_ENGINE}")
print(f"Versión de pandas: {pd.__version__}")


Engine de lectura seleccionado: pyarrow
Versión de pandas: 2.3.0


In [2]:

# %% [optional unzip]
# Si has subido un ZIP con la carpeta: data_drive.zip, lo descomprimimos aquí.
from zipfile import ZipFile

zip_path = Path("data_drive.zip")
if zip_path.exists():
    print("Detectado data_drive.zip -> descomprimiendo...")
    with ZipFile(zip_path, "r") as zf:
        zf.extractall(".")
    print("Descompresión completada.")
else:
    print("No se encontró data_drive.zip (no es obligatorio si ya existe la carpeta data_drive/)")


No se encontró data_drive.zip (no es obligatorio si ya existe la carpeta data_drive/)


In [3]:

# %% [config]
# Ruta raíz donde están las subcarpetas LS1..LS6
RAIZ = Path("data_drive")

assert RAIZ.exists() and RAIZ.is_dir(), (
    f"No se encontró la carpeta '{RAIZ.resolve()}'.\n"
    "Coloca aquí tu carpeta descargada desde Drive o sube 'data_drive.zip' y vuelve a ejecutar la celda anterior."
)

print("Directorio raíz:", RAIZ.resolve())


Directorio raíz: D:\ls_feed\data_drive


In [4]:

# %% [funciones]
def listar_dirs_ls(raiz: Path) -> List[Path]:
    patron = re.compile(r"^LS([1-6])(?:\b|[_\-].*)?$", flags=re.IGNORECASE)
    dirs = [p for p in raiz.iterdir() if p.is_dir() and patron.match(p.name)]
    return sorted(dirs, key=lambda x: x.name.lower())

def leer_csv_con_robustez(path_csv: Path) -> pd.DataFrame:
    posibles_sep = [",", ";", "\t", "|"]
    for sep in posibles_sep:
        try:
            df = pd.read_csv(path_csv, sep=sep, engine=_ENGINE)
            df["source_file"] = str(path_csv)
            return df
        except Exception:
            pass
    for sep in posibles_sep:
        try:
            df = pd.read_csv(path_csv, sep=sep, engine="python")
            df["source_file"] = str(path_csv)
            return df
        except Exception:
            pass
    raise RuntimeError(f"No se pudo leer el CSV: {path_csv}")

def concat_csvs_de_directorio(dir_ls: Path) -> Optional[pd.DataFrame]:
    csvs = sorted(dir_ls.rglob("*.csv"))
    if not csvs:
        logging.warning("Sin CSVs en %s", dir_ls)
        return None
    frames = []
    for csv_path in csvs:
        try:
            frames.append(leer_csv_con_robustez(csv_path))
        except Exception as e:
            logging.error("Error leyendo %s: %s", csv_path, e)
    if not frames:
        return None
    df = pd.concat(frames, ignore_index=True, copy=False)
    return df

def construir_dataframes_por_ls(raiz_descarga: Path) -> Dict[str, pd.DataFrame]:
    res: Dict[str, pd.DataFrame] = {}
    for dir_ls in listar_dirs_ls(raiz_descarga):
        m = re.match(r"^LS([1-6])", dir_ls.name, flags=re.IGNORECASE)
        if not m:
            continue
        idx = m.group(1)
        df = concat_csvs_de_directorio(dir_ls)
        if df is not None:
            res[f"ls{idx}"] = df
    return res

def exponer_variables_globales(dfs_por_ls: Dict[str, pd.DataFrame]) -> None:
    for clave, df in dfs_por_ls.items():
        var_name = f"df_{clave}"
        globals()[var_name] = df
        logging.info("Creada variable: %s  shape=%s", var_name, df.shape)


In [5]:

# %% [ejecución principal]
dfs_por_ls = construir_dataframes_por_ls(RAIZ)
exponer_variables_globales(dfs_por_ls)

if not dfs_por_ls:
    raise SystemExit("No se encontraron subcarpetas LS con CSVs bajo la ruta indicada.")

# Resumen
resumen = {k: v.shape for k, v in dfs_por_ls.items()}
resumen


14:15:48 | INFO | Creada variable: df_ls1  shape=(1059, 2)
14:15:48 | INFO | Creada variable: df_ls2  shape=(903, 2)
14:15:48 | INFO | Creada variable: df_ls3  shape=(1057, 2)
14:15:48 | INFO | Creada variable: df_ls4  shape=(1057, 2)
14:15:48 | INFO | Creada variable: df_ls5  shape=(1059, 2)
14:15:48 | INFO | Creada variable: df_ls6  shape=(16, 2)


{'ls1': (1059, 2),
 'ls2': (903, 2),
 'ls3': (1057, 2),
 'ls4': (1057, 2),
 'ls5': (1059, 2),
 'ls6': (16, 2)}

In [6]:

# %% [previews]
from itertools import islice

for clave, df in dfs_por_ls.items():
    print(f"\n=== Vista previa df_{clave} ===")
    display(df.head(10))



=== Vista previa df_ls1 ===


,mac;time;vlx,source_file
0,40:22:D8:F1:E3:70;2025/07/15 11:44:19;316.00,data_drive\LS1\silos_vlx_1.csv
1,40:22:D8:F1:E3:70;2025/07/15 12:14:21;317.00,data_drive\LS1\silos_vlx_1.csv
2,40:22:D8:F1:E3:70;2025/07/15 12:44:22;326.00,data_drive\LS1\silos_vlx_1.csv
3,40:22:D8:F1:E3:70;2025/07/15 13:14:24;315.00,data_drive\LS1\silos_vlx_1.csv
4,40:22:D8:F1:E3:70;2025/07/15 13:44:26;316.00,data_drive\LS1\silos_vlx_1.csv
5,40:22:D8:F1:E3:70;2025/07/15 14:14:27;308.00,data_drive\LS1\silos_vlx_1.csv
6,40:22:D8:F1:E3:70;2025/07/15 14:44:29;309.00,data_drive\LS1\silos_vlx_1.csv
7,40:22:D8:F1:E3:70;2025/07/15 15:14:30;309.00,data_drive\LS1\silos_vlx_1.csv
8,40:22:D8:F1:E3:70;2025/07/15 15:44:32;310.00,data_drive\LS1\silos_vlx_1.csv
9,40:22:D8:F1:E3:70;2025/07/15 16:14:33;316.00,data_drive\LS1\silos_vlx_1.csv



=== Vista previa df_ls2 ===


,mac;time;vlx,source_file
0,40:22:D8:F1:E2:CC;2025/07/15 11:44:46;272.00,data_drive\LS2\silos_vlx_0.csv
1,40:22:D8:F1:E2:CC;2025/07/15 12:14:48;283.00,data_drive\LS2\silos_vlx_0.csv
2,40:22:D8:F1:E2:CC;2025/07/15 12:44:49;283.00,data_drive\LS2\silos_vlx_0.csv
3,40:22:D8:F1:E2:CC;2025/07/15 13:14:51;281.00,data_drive\LS2\silos_vlx_0.csv
4,40:22:D8:F1:E2:CC;2025/07/15 13:44:53;278.00,data_drive\LS2\silos_vlx_0.csv
5,40:22:D8:F1:E2:CC;2025/07/15 14:14:54;279.00,data_drive\LS2\silos_vlx_0.csv
6,40:22:D8:F1:E2:CC;2025/07/15 14:44:56;285.00,data_drive\LS2\silos_vlx_0.csv
7,40:22:D8:F1:E2:CC;2025/07/15 15:14:57;281.00,data_drive\LS2\silos_vlx_0.csv
8,40:22:D8:F1:E2:CC;2025/07/15 15:44:59;277.00,data_drive\LS2\silos_vlx_0.csv
9,40:22:D8:F1:E2:CC;2025/07/15 16:15:00;276.00,data_drive\LS2\silos_vlx_0.csv



=== Vista previa df_ls3 ===


...

,mac;time;vlx,source_file
0,B8:D6:1A:60:95:30;2024/06/14 00:01:00;337.00,data_drive\LS4\silos_vlx_0.csv
1,B8:D6:1A:60:95:30;2024/06/14 00:01:02;349.00,data_drive\LS4\silos_vlx_0.csv
2,B8:D6:1A:60:95:30;2024/06/14 00:01:00;352.00,data_drive\LS4\silos_vlx_0.csv
3,B8:D6:1A:60:95:30;2024/06/14 00:01:01;354.00,data_drive\LS4\silos_vlx_0.csv
4,B8:D6:1A:60:95:30;2024/06/14 00:01:03;344.00,data_drive\LS4\silos_vlx_0.csv
5,B8:D6:1A:60:95:30;2024/06/14 00:01:00;342.00,data_drive\LS4\silos_vlx_0.csv
6,B8:D6:1A:60:95:30;2024/06/14 00:01:00;346.00,data_drive\LS4\silos_vlx_0.csv
7,B8:D6:1A:60:95:30;2024/06/14 00:01:01;346.00,data_drive\LS4\silos_vlx_0.csv
8,B8:D6:1A:60:95:30;2024/06/14 00:01:03;349.00,data_drive\LS4\silos_vlx_0.csv
9,B8:D6:1A:60:95:30;2024/06/14 00:01:00;345.00,data_drive\LS4\silos_vlx_0.csv



=== Vista previa df_ls5 ===


,mac;time;vlx,source_file
0,B8:D6:1A:60:94:1C;2025/07/15 11:17:12;328.00,data_drive\LS5\silos_vlx_0.csv
1,B8:D6:1A:60:94:1C;2025/07/15 11:47:13;330.00,data_drive\LS5\silos_vlx_0.csv
2,B8:D6:1A:60:94:1C;2025/07/15 12:17:15;333.00,data_drive\LS5\silos_vlx_0.csv
3,B8:D6:1A:60:94:1C;2025/07/15 12:47:17;339.00,data_drive\LS5\silos_vlx_0.csv
4,B8:D6:1A:60:94:1C;2025/07/15 13:17:18;338.00,data_drive\LS5\silos_vlx_0.csv
5,B8:D6:1A:60:94:1C;2025/07/15 13:47:20;342.00,data_drive\LS5\silos_vlx_0.csv
6,B8:D6:1A:60:94:1C;2025/07/15 14:17:21;335.00,data_drive\LS5\silos_vlx_0.csv
7,B8:D6:1A:60:94:1C;2025/07/15 14:47:23;335.00,data_drive\LS5\silos_vlx_0.csv
8,B8:D6:1A:60:94:1C;2025/07/15 15:17:24;334.00,data_drive\LS5\silos_vlx_0.csv
9,B8:D6:1A:60:94:1C;2025/07/15 15:47:26;331.00,data_drive\LS5\silos_vlx_0.csv



=== Vista previa df_ls6 ===


,mac;time;vlx,source_file
0,D8:13:2A:D2:36:B4;2025/07/15 11:17:11;364.00,data_drive\LS6\silos_vlx_0.csv
1,D8:13:2A:D2:36:B4;2025/07/15 11:33:22;374.00,data_drive\LS6\silos_vlx_1.csv
2,D8:13:2A:D2:36:B4;2025/07/30 17:58:37;373.00,data_drive\LS6\silos_vlx_10.csv
3,D8:13:2A:D2:36:B4;2025/07/30 18:08:37;381.00,data_drive\LS6\silos_vlx_11.csv
4,D8:13:2A:D2:36:B4;2025/07/30 18:18:37;391.00,data_drive\LS6\silos_vlx_12.csv
5,D8:13:2A:D2:36:B4;2025/08/06 11:08:39;236.00,data_drive\LS6\silos_vlx_13.csv
6,D8:13:2A:D2:36:B4;2025/08/06 11:20:27;230.00,data_drive\LS6\silos_vlx_14.csv
7,D8:13:2A:D2:36:B4;2025/08/06 11:34:13;229.00,data_drive\LS6\silos_vlx_15.csv
8,D8:13:2A:D2:36:B4;2025/07/15 11:46:09;371.00,data_drive\LS6\silos_vlx_2.csv
9,D8:13:2A:D2:36:B4;2025/07/15 11:58:16;378.00,data_drive\LS6\silos_vlx_3.csv


14:15:48 | WARNING | Cabecera inesperada: ['40:22:D8:F1:E3:70', '2025/07/15 11:44:19', '316.00'] -> se usará igualmente
14:15:48 | INFO | Procesado: ls1.parquet | primera_col=mac;time;vlx | shape (1059, 2) -> (1058, 4)
14:15:48 | WARNING | Cabecera inesperada: ['40:22:D8:F1:E2:CC', '2025/07/15 11:44:46', '272.00'] -> se usará igualmente
14:15:48 | INFO | Procesado: ls2.parquet | primera_col=mac;time;vlx | shape (903, 2) -> (902, 4)
14:15:48 | WARNING | Cabecera inesperada: ['40:22:D8:F1:E3:80', '2025/07/15 12:07:07', '286.00'] -> se usará igualmente
14:15:48 | INFO | Procesado: ls3.parquet | primera_col=mac;time;vlx | shape (1057, 2) -> (1056, 4)
14:15:48 | WARNING | Cabecera inesperada: ['B8:D6:1A:60:95:30', '2024/06/14 00:01:00', '337.00'] -> se usará igualmente
14:15:48 | INFO | Procesado: ls4.parquet | primera_col=mac;time;vlx | shape (1057, 2) -> (1056, 4)
14:15:48 | WARNING | Cabecera inesperada: ['B8:D6:1A:60:94:1C', '2025/07/15 11:17:12', '328.00'] -> se usará igualmente
14:15:


Resumen de archivos transformados:
- ls1.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (1059, 2) -> (1058, 4) | guardado en: data_drive\_parquet_export\split\ls1.parquet
- ls2.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (903, 2) -> (902, 4) | guardado en: data_drive\_parquet_export\split\ls2.parquet
- ls3.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (1057, 2) -> (1056, 4) | guardado en: data_drive\_parquet_export\split\ls3.parquet
- ls4.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (1057, 2) -> (1056, 4) | guardado en: data_drive\_parquet_export\split\ls4.parquet
- ls5.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (1059, 2) -> (1058, 4) | guardado en: data_drive\_parquet_export\split\ls5.parquet
- ls6.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (16, 2) -> (15, 4) | guardado en: data_drive\_parquet_export\split\ls6.parquet


In [8]:
# %% Separar primera columna en columnas mac, time, vlx en todos los .parquet
from __future__ import annotations
from pathlib import Path
import pandas as pd
import logging

# Configuración de rutas
IN_DIR = Path("data_drive/_parquet_export")   # carpeta donde están los .parquet originales
OUT_DIR = IN_DIR / "split"                    # salida con los .parquet transformados
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%H:%M:%S",
)

def split_first_col_to_mac_time_vlx(df: pd.DataFrame, sep: str = ";") -> pd.DataFrame:
    """
    Convierte la primera columna del DataFrame en 3 columnas fijas: mac, time, vlx.
    - Ignora el valor de cabecera embebido (ej. 'mac;time;vlx') en la primera fila.
    - Elimina esa primera fila, ya que no contiene datos reales.
    - Devuelve un DF con columnas: mac, time, vlx (+ source_file si existe).
    """
    if df.shape[1] < 2:
        raise ValueError("Se esperaba al menos 2 columnas: la combinada y source_file.")

    first_col = df.columns[0]
    s = df[first_col].astype("string")

    # Separar toda la columna en 3 partes
    parts = s.str.split(sep, n=2, expand=True)
    parts.columns = ["mac", "time", "vlx"]

    # Eliminar fila de cabecera
    parts = parts.iloc[1:].reset_index(drop=True)
    df_rest = df.drop(index=0).reset_index(drop=True)

    # Ensamblar: mac, time, vlx + source_file (si existe)
    if "source_file" in df_rest.columns:
        out = pd.concat([parts, df_rest[["source_file"]]], axis=1)
    else:
        out = parts

    return out

# Procesamiento masivo
parquets = sorted(IN_DIR.rglob("*.parquet"))
if not parquets:
    raise SystemExit(f"No se encontraron .parquet en {IN_DIR.resolve()}")

resumen = []
for p in parquets:
    try:
        df = pd.read_parquet(p)
        first_col = df.columns[0] if df.shape[1] else "<sin_columnas>"
        df_out = split_first_col_to_mac_time_vlx(df, sep=";")
        out_path = OUT_DIR / p.name
        df_out.to_parquet(out_path, index=False)
        resumen.append((p.name, first_col, df.shape, df_out.shape, out_path))
        logging.info("Procesado: %s | primera_col=%s | shape %s -> %s",
                     p.name, first_col, df.shape, df_out.shape)
    except Exception as e:
        logging.error("Error procesando %s: %s", p, e)

# Resumen final
print("\nResumen de archivos transformados:")
for name, first_col, in_shape, out_shape, out_path in resumen:
    print(f"- {name}: '{first_col}' -> columnas [mac, time, vlx, source_file] | {in_shape} -> {out_shape} | guardado en: {out_path}")


14:17:49 | INFO | Procesado: ls1.parquet | primera_col=mac;time;vlx | shape (1059, 2) -> (1058, 4)
14:17:49 | INFO | Procesado: ls2.parquet | primera_col=mac;time;vlx | shape (903, 2) -> (902, 4)
14:17:49 | INFO | Procesado: ls3.parquet | primera_col=mac;time;vlx | shape (1057, 2) -> (1056, 4)
14:17:49 | INFO | Procesado: ls4.parquet | primera_col=mac;time;vlx | shape (1057, 2) -> (1056, 4)
14:17:49 | INFO | Procesado: ls5.parquet | primera_col=mac;time;vlx | shape (1059, 2) -> (1058, 4)
14:17:49 | INFO | Procesado: ls6.parquet | primera_col=mac;time;vlx | shape (16, 2) -> (15, 4)
14:17:49 | ERROR | Error procesando data_drive\_parquet_export\split\ls1.parquet: Length mismatch: Expected axis has 1 elements, new values have 3 elements
14:17:49 | ERROR | Error procesando data_drive\_parquet_export\split\ls2.parquet: Length mismatch: Expected axis has 1 elements, new values have 3 elements
14:17:49 | ERROR | Error procesando data_drive\_parquet_export\split\ls3.parquet: Length mismatch: E


Resumen de archivos transformados:
- ls1.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (1059, 2) -> (1058, 4) | guardado en: data_drive\_parquet_export\split\ls1.parquet
- ls2.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (903, 2) -> (902, 4) | guardado en: data_drive\_parquet_export\split\ls2.parquet
- ls3.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (1057, 2) -> (1056, 4) | guardado en: data_drive\_parquet_export\split\ls3.parquet
- ls4.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (1057, 2) -> (1056, 4) | guardado en: data_drive\_parquet_export\split\ls4.parquet
- ls5.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (1059, 2) -> (1058, 4) | guardado en: data_drive\_parquet_export\split\ls5.parquet
- ls6.parquet: 'mac;time;vlx' -> columnas [mac, time, vlx, source_file] | (16, 2) -> (15, 4) | guardado en: data_drive\_parquet_export\split\ls6.parquet
